# Build Simple State Machine with PyTransitions

## 1. State Diagram

```lua
stateDiagram-v2
    INIT --> GENERATE
    GENERATE --> FINAL
```

```mermaid
stateDiagram-v2
    INIT --> GENERATE
    GENERATE --> FINAL
```


---

## 2. Use PyTransitions

In [1]:
import asyncio
from transitions.extensions.asyncio import AsyncMachine
from openai import AsyncOpenAI
client = AsyncOpenAI()

async def generate(messages, agent_name, user_message):
    
    # Execute
    
    messages.append({"role": "system", "content": f"Your name is {agent_name}. You are a helpful assistant."})
    messages.append({"role": "user", "content": f"{agent_name}, {user_message}"})
    response = await client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
        max_tokens=50,
        stream=True
    )
    
    async def streamer():
        content = ""
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                content += chunk
                yield chunk
        messages.append({
            "role": "assistant",
            "content": content
        })

    return streamer()

class AsyncStateMachine:
    
    def __init__(self,name):
        self.states = ["INIT", "GENERATE", "FINAL"]
        self.machine = AsyncMachine(model=self, states=self.states, initial="INIT")
        self.machine.add_transition(trigger="run_async",source="INIT",dest="GENERATE", after="after_action")
        self.machine.add_transition(trigger="run_async",source="GENERATE",dest="FINAL")
        self.monologue_messages = []
        self.name = name
        self.user_message = "Hello, world!"
    
    async def after_action(self):        
        self.streamer = await generate(
            messages=self.monologue_messages, 
            agent_name=self.name,
            user_message=self.user_message
            )

asm = AsyncStateMachine("Sara")
await asm.run_async()
async for chunk in asm.streamer:
    print(chunk, end='', flush=True)

Hello, world! 👋 How can I help you today?

In [2]:
print(asm.monologue_messages)

[{'role': 'system', 'content': 'Your name is Sara. You are a helpful assistant.'}, {'role': 'user', 'content': 'Sara, Hello, world!'}, {'role': 'assistant', 'content': 'Hello, world! 👋 How can I help you today?'}]
